In [1]:
import psycopg2, time
import numpy as np
from os import environ, chdir; chdir('../')
conn = psycopg2.connect(dbname='postgres', 
                        user='postgres', 
                        host=environ['REDHAT_POSTGRES_1_PORT_5432_TCP_ADDR'])
cur = conn.cursor()

In [2]:
from lib.app import Q
from lib.helpers.database_helper import pull_and_shape_batch
from lib.helpers.redis_helper import read_weights_matrix_from_redis, write_weights_matrix_to_redis
from lib.nn.functions import loss_function, scores


## Random Search

In [ ]:
bestloss = float("inf") # Python assigns the highest possible float value

random_W = np.random.randn(10, 7326) * 0.0001 
result, (l,w) = write_weights_matrix_to_redis(random_W)

for num in range(25):
    t = time.time()
    batch_features, \
        batch_outcomes = pull_and_shape_batch(100,num*100)
        
    # generate random parameters
    random_W = np.random.randn(10, 7326) * 0.0001 
    
    # get the loss over the entire training set
    loss = loss_function(random_W, batch_features, batch_outcomes) 
    
    if loss < bestloss: # keep track of the best solution
        bestloss = loss
        bestW = random_W
        write_weights_matrix_to_redis(bestW)
    print("""in attempt {:2} the loss was {:.4f} best {:.4f} took {:.4f} seconds""".format(num, 
                                                                             loss, 
                                                                             bestloss, 
                                                                             time.time() - t))

in attempt  0 the loss was 4.5021 best 4.5021 took 0.9815 seconds
in attempt  1 the loss was 4.5069 best 4.5021 took 1.0498 seconds
in attempt  2 the loss was 4.5007 best 4.5007 took 1.1534 seconds
in attempt  3 the loss was 4.4995 best 4.4995 took 4.7892 seconds
in attempt  4 the loss was 4.5325 best 4.4995 took 9.9807 seconds
in attempt  5 the loss was 4.5361 best 4.4995 took 0.9537 seconds
in attempt  6 the loss was 4.5261 best 4.4995 took 0.9164 seconds
in attempt  7 the loss was 4.4971 best 4.4971 took 0.9407 seconds
in attempt  8 the loss was 4.5148 best 4.4971 took 0.9873 seconds
in attempt  9 the loss was 4.6100 best 4.4971 took 0.9104 seconds
